## Recommedation Engine

In [33]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split

import scipy.sparse as sparse

from scipy.sparse.linalg import spsolve
import warnings
warnings.filterwarnings('ignore')
import sys

# Step 1 - Data Import FactInternet Sales/Product/Customer/Geophraphy SQl Table

In [2]:
import pyodbc 
import pandas as pd
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-QEC24C4;'
                      'Database=AdventureWorksDW2017;' 
                      'Trusted_Connection=yes;')


sql_for_df1 = """SELECT
FI.[SalesOrderNumber],
FI.[ProductKey] as ProductID,
P.EnglishProductName as ProductName,
FI.[CustomerKey] as CustomerID,
FI.[OrderQuantity] as OrderVol,
C.[BirthDate],
C.[MaritalStatus],
C.[Gender],
C.[YearlyIncome],
C.[TotalChildren],
C.[HouseOwnerFlag],
C.[NumberCarsOwned],
C.[CommuteDistance],
G.[City],
G.[CountryRegionCode]
FROM [dbo].[FactInternetSales] as FI
LEFT JOIN [dbo].[DimProduct] as P
ON FI.ProductKey=P.ProductKey 
LEFT JOIN [dbo].[DimCustomer] AS C
ON FI.[CustomerKey]= C.[CustomerKey]
LEFT JOIN [dbo].[DimGeography] AS G
ON C.[GeographyKey] = G.[GeographyKey]"""

df = pd.read_sql(sql_for_df1 , conn)


conn.close()
df.head()

,SalesOrderNumber,ProductID,ProductName,CustomerID,OrderVol,BirthDate,MaritalStatus,Gender,YearlyIncome,TotalChildren,HouseOwnerFlag,NumberCarsOwned,CommuteDistance,City,CountryRegionCode
0,SO43697,310,"Road-150 Red, 62",21768,1,1952-02-19,S,M,70000.0,5,1,3,10+ Miles,Metchosin,CA
1,SO43698,346,"Mountain-100 Silver, 44",28389,1,1970-06-17,S,F,20000.0,3,0,0,0-1 Miles,Pantin,FR
2,SO43699,346,"Mountain-100 Silver, 44",25863,1,1952-06-01,S,F,40000.0,5,1,3,10+ Miles,Lebanon,US
3,SO43700,336,"Road-650 Black, 62",14501,1,1943-11-10,M,M,80000.0,4,1,2,1-2 Miles,Beverly Hills,US
4,SO43701,346,"Mountain-100 Silver, 44",11003,1,1973-08-14,S,F,70000.0,0,0,1,5-10 Miles,North Ryde,AU


## Data Exploration

In [82]:
df.shape #checkings statistics

(60398, 15)

In [83]:
# Check for missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SalesOrderNumber   60398 non-null  object 
 1   ProductID          60398 non-null  int64  
 2   ProductName        60398 non-null  object 
 3   CustomerID         60398 non-null  int32  
 4   OrderVol           60398 non-null  int64  
 5   BirthDate          60398 non-null  object 
 6   MaritalStatus      60398 non-null  object 
 7   Gender             60398 non-null  object 
 8   YearlyIncome       60398 non-null  float64
 9   TotalChildren      60398 non-null  int64  
 10  HouseOwnerFlag     60398 non-null  object 
 11  NumberCarsOwned    60398 non-null  int64  
 12  CommuteDistance    60398 non-null  object 
 13  City               60398 non-null  object 
 14  CountryRegionCode  60398 non-null  object 
dtypes: float64(1), int32(1), int64(4), object(9)
memory usage: 6.7+ MB


## Importing Product dataset

In [84]:
#Importing Product dataset
import pyodbc 
import pandas as pd
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-QEC24C4;'
                      'Database=AdventureWorksDW2017;' 
                      'Trusted_Connection=yes;')



sql_for_df2 = """SELECT
* FROM [dbo].[DimProduct] """

df2 = pd.read_sql(sql_for_df2 , conn)
df2.head()


,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,...,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,GermanDescription,JapaneseDescription,TurkishDescription,StartDate,EndDate,Status
0,1,AR-5381,NaN,None,None,Adjustable Race,,,NaN,False,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
1,2,BA-8327,NaN,None,None,Bearing Ball,,,NaN,False,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
2,3,BE-2349,NaN,None,None,BB Ball Bearing,,,NaN,False,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
3,4,BE-2908,NaN,None,None,Headset Ball Bearings,,,NaN,False,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
4,5,BL-2036,NaN,None,None,Blade,,,NaN,False,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current


In [6]:
df2.shape

(606, 36)

In [7]:
df2.dtypes

ProductKey                        int64
ProductAlternateKey              object
ProductSubcategoryKey           float64
WeightUnitMeasureCode            object
SizeUnitMeasureCode              object
EnglishProductName               object
SpanishProductName               object
FrenchProductName                object
StandardCost                    float64
FinishedGoodsFlag                  bool
Color                            object
SafetyStockLevel                  int64
ReorderPoint                      int64
ListPrice                       float64
Size                             object
SizeRange                        object
Weight                          float64
DaysToManufacture                 int64
ProductLine                      object
DealerPrice                     float64
Class                            object
Style                            object
ModelName                        object
LargePhoto                       object
EnglishDescription               object


In [8]:
df2.describe()

,ProductKey,ProductSubcategoryKey,StandardCost,SafetyStockLevel,ReorderPoint,ListPrice,Weight,DaysToManufacture,DealerPrice
count,606.00000,397.000000,395.000000,606.000000,606.000000,395.000000,282.000000,606.000000,395.000000
mean,303.50000,12.128463,434.265829,495.201320,371.400990,747.661762,56.193227,1.201320,448.597060
std,175.08141,9.423328,497.379884,364.007114,273.005336,838.708451,158.000652,1.508893,503.225071
min,1.00000,1.000000,0.856500,4.000000,3.000000,2.290000,2.120000,0.000000,1.374000
25%,152.25000,2.000000,37.120900,100.000000,75.000000,69.990000,2.680000,0.000000,41.994000
50%,303.50000,13.000000,204.625100,500.000000,375.000000,364.090000,15.420000,1.000000,218.454000
75%,454.75000,16.000000,660.914200,1000.000000,750.000000,1204.324800,24.130000,1.000000,722.594900
max,606.00000,37.000000,2171.294200,1000.000000,750.000000,3578.270000,1050.000000,4.000000,2146.962000


In [9]:
df2.isnull().sum()

ProductKey                 0
ProductAlternateKey        0
ProductSubcategoryKey    209
WeightUnitMeasureCode    324
SizeUnitMeasureCode      353
EnglishProductName         0
SpanishProductName         0
FrenchProductName          0
StandardCost             211
FinishedGoodsFlag          0
Color                      0
SafetyStockLevel           0
ReorderPoint               0
ListPrice                211
Size                     307
SizeRange                  0
Weight                   324
DaysToManufacture          0
ProductLine              226
DealerPrice              211
Class                    276
Style                    305
ModelName                209
LargePhoto                 0
EnglishDescription       210
FrenchDescription        210
ChineseDescription       210
ArabicDescription        210
HebrewDescription        210
ThaiDescription          210
GermanDescription        210
JapaneseDescription      210
TurkishDescription       210
StartDate                  0
EndDate       

In [10]:
df2.rename(columns={'ProductKey':'ProductID'}, inplace=True)

## Importing subcategories table from SQL

In [11]:
# Importing subcategories table from SQL
import pyodbc 
import pandas as pd
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-QEC24C4;'
                      'Database=AdventureWorksDW2017;' 
                      'Trusted_Connection=yes;')



sql_for_df3 = """SELECT
* FROM [dbo].[DimProductSubcategory] """

df3 = pd.read_sql(sql_for_df3 , conn)
df3.head()




,ProductSubcategoryKey,ProductSubcategoryAlternateKey,EnglishProductSubcategoryName,SpanishProductSubcategoryName,FrenchProductSubcategoryName,ProductCategoryKey
0,1,1,Mountain Bikes,Bicicleta de montaña,VTT,1
1,2,2,Road Bikes,Bicicleta de carretera,Vélo de route,1
2,3,3,Touring Bikes,Bicicleta de paseo,Vélo de randonnée,1
3,4,4,Handlebars,Barra,Barre d'appui,2
4,5,5,Bottom Brackets,Eje de pedalier,Axe de pédalier,2


In [12]:
df3.shape

(37, 6)

In [13]:
#add product sub category info to product data
allproducts=df2.merge(df3,on='ProductSubcategoryKey',how='left')

In [14]:
print(allproducts.shape)

(606, 41)


### Recommendation Engine User Filter

In [15]:
#choose one method below:

internet_sales=df #no filtering
#internet_sales=internet_sales_alldata.loc[internet_sales_alldata['CountryRegionCode'] == 'CA'] # geo filter
#internet_sales=internet_sales_alldata.loc[internet_sales_alldata['Gender'] == 'F'] # gender filter
#internet_sales=internet_sales_alldata.loc[(internet_sales_alldata['CountryRegionCode'] == 'CA') & (internet_sales_alldata['YearlyIncome'] > 40000)] #dual filter

In [16]:
#checks to see if filtering worked
internet_sales.shape # the number should correspond with the ttoal count in SQL whic is 60398

(60398, 15)

In [17]:
internet_sales.dtypes

SalesOrderNumber      object
ProductID              int64
ProductName           object
CustomerID             int64
OrderVol               int64
BirthDate             object
MaritalStatus         object
Gender                object
YearlyIncome         float64
TotalChildren          int64
HouseOwnerFlag        object
NumberCarsOwned        int64
CommuteDistance       object
City                  object
CountryRegionCode     object
dtype: object

In [18]:
internet_sales.describe()

,ProductID,CustomerID,OrderVol,YearlyIncome,TotalChildren,NumberCarsOwned
count,60398.000000,60398.000000,60398.0,60398.000000,60398.00000,60398.000000
mean,437.557932,18841.685420,1.0,59715.056790,1.85074,1.509189
std,118.088390,5432.430404,0.0,33065.426837,1.62107,1.154163
min,214.000000,11000.000000,1.0,10000.000000,0.00000,0.000000
25%,359.000000,14003.000000,1.0,30000.000000,0.00000,1.000000
50%,479.000000,18143.000000,1.0,60000.000000,2.00000,2.000000
75%,529.000000,23429.750000,1.0,80000.000000,3.00000,2.000000
max,606.000000,29483.000000,1.0,170000.000000,5.00000,4.000000


In [19]:
# Now, we create a product look up table that keep tracks of each product ID along with its name
product_lookup = internet_sales[['ProductID', 'ProductName']].drop_duplicates() # Only get unique product/description pairs
product_lookup['ProductID'] = product_lookup.ProductID.astype(str) # Encode as strings for future lookup ease

In [20]:
product_lookup.head()

,ProductID,ProductName
0,310,"Road-150 Red, 62"
1,346,"Mountain-100 Silver, 44"
3,336,"Road-650 Black, 62"
5,311,"Road-150 Red, 44"
7,351,"Mountain-100 Black, 48"


In [21]:
product_lookup.dtypes 

ProductID      object
ProductName    object
dtype: object

In [22]:
product_lookup.shape

(158, 2)

In [23]:
#*check* there are 130 unique products, but 158 product mappings in the dataset
#due to variations in standard cost/list price of some products 
product_lookup.describe()

,ProductID,ProductName
count,158,158
unique,158,130
top,310,"Road-550-W Yellow, 44"
freq,1,2


In [24]:
#now prepare the dataframe for modelling later - we want customerID, ProductID and OrderVol only

internet_sales['CustomerID'] = internet_sales.CustomerID.astype(int) # Convert to int for customer ID
internet_sales = internet_sales[['ProductID', 'CustomerID', 'OrderVol']] # Focus only on ProductID, CustomerID and OrderVol
grouped_cleaned = internet_sales.groupby(['CustomerID', 'ProductID']).sum().reset_index() # Group together 
#*check above should still maintain same number of rows of original dataset
#*check also - might want to name 'grouped' only, or 'reduced' as hasn't really been 'cleaned'

In [25]:
grouped_cleaned.head()

,CustomerID,ProductID,OrderVol
0,11000,214,1
1,11000,344,1
2,11000,353,1
3,11000,485,1
4,11000,488,1


In [26]:
#check no zero or negative purchases
grouped_cleaned.loc[grouped_cleaned.OrderVol <= 0]

,CustomerID,ProductID,OrderVol


In [27]:
grouped_cleaned.info()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59051 entries, 0 to 59050
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   CustomerID  59051 non-null  int64
 1   ProductID   59051 non-null  int64
 2   OrderVol    59051 non-null  int64
dtypes: int64(3)
memory usage: 1.4 MB


In [28]:
grouped_cleaned['ProductID'] = grouped_cleaned['ProductID'].astype(str) #convert ProductID to string
grouped_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59051 entries, 0 to 59050
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerID  59051 non-null  int64 
 1   ProductID   59051 non-null  object
 2   OrderVol    59051 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


## Sparse Confidence Matrix

In [29]:
# We don't have explicit customer "ratings" (cf Netflix/Amazon movie/product ratings), but the order volume can represent a "confidence" in terms 
# of how strong the interaction was.

### Create a sparse confidence matrix of customers and products

In [30]:

customers = list(grouped_cleaned.CustomerID.unique()) # Get our unique customers
products = list(grouped_cleaned.ProductID.unique()) # Get our unique products that were purchased
quantity = list(grouped_cleaned.OrderVol) # All of our purchases



In [35]:
from pandas.api.types import CategoricalDtype
#get the associated row indices
rows= grouped_cleaned.CustomerID.astype(CategoricalDtype(categories=customers)).cat.codes
#get the associated column indices
cols= grouped_cleaned.ProductID.astype(CategoricalDtype(categories=products)).cat.codes
#compress sparse row matrix
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [36]:
#more checks
print(len(customers))
print(len(products))

18484
158


In [37]:
purchases_sparse

<18484x158 sparse matrix of type '<class 'numpy.intc'>'
	with 59051 stored elements in Compressed Sparse Row format>

In [ ]:
#a sparse matrix is not a pandas dataframe, but sparse matrices are efficient for row slicing and fast

In [38]:
# Based on the unfiltered data set, we have 18484 unique customers with 158 unique products.
#For these customer/product interactions, 59051 of these products had a purchase. In terms of sparsity of the matrix, that makes:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of products interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

97.97803231806365

-97.97% of the interaction matrix is sparse. For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. We are well below this, so we should be able to get decent results.

### Training/validating customer/product interactions - matrix factorisation

- Typically in Machine Learning applications, we need to test whether the model we just trained is any good on new data it hasn’t yet seen before from the training phase. We do this by creating a test set completely separate from the training set. Usually this is fairly simple: just take a random sample of the training example rows in our feature matrix and separate it away from the training set.
- With collaborative filtering, that’s not going to work because you need all of the user/item interactions to find the proper matrix factorization. A better method is to hide a certain percentage of the user/item interactions from the model during the training phase chosen at random. Then, check during the test phase how many of the items that were recommended the user actually ended up purchasing in the end. Ideally, you would ultimately test your recommendations with some kind of A/B test or utilizing data from a time series where all data prior to a certain point in time is used for training while data after a certain period of time is used for testing.
- Our test set is an exact copy of our original data. The training set, however, will mask a random percentage of user/item interactions and act as if the user never purchased the item (making it a sparse entry with a zero). We then check in the test set which items were recommended to the user that they ended up actually purchasing. If the users frequently ended up purchasing the items most recommended to them by the system, we can conclude the system seems to be working.
- Now that we have a plan on how to separate our training and testing sets, let’s create a function that can do this for us. We will also import the random library and set a seed so that you will see the same results as I did.

In [39]:
# We'll use a random function to separate our training and testing sets.
import random

In [40]:
def make_train(confidence, pct_test = 0.2):
    '''
    This function will take in the original customer-product matrix and "mask" a percentage of the original confidence where a
    customer-product interaction has taken place for use as a test set. The test set will contain all of the original confidence, 
    while the training set replaces the specified percentage of them with a zero in the original confidence matrix. 
    
    parameters: 
    
    confidence - the original confidence matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of customer-product interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original confidence. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the customer-product pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original confidence matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    customer_inds - From the randomly selected customer-product indices, which customer rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = confidence.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = confidence.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the confidence data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of customer,product index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of customer-product pairs without replacement
    customer_inds = [index[0] for index in samples] # Get the customer row indices
    product_inds = [index[1] for index in samples] # Get the product column indices
    training_set[customer_inds, product_inds] = 0 # Assign all of the randomly chosen customer-product pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(customer_inds)) # Output the unique list of customer rows that were altered

-This will return our training set, a test set that has been binarized to 0/1 for purchased/not purchased,and a list of which users had at least one item masked. We will test the performance of the recommender system on these users only. 
-I am masking 20% of the user/item interactions for this project.

In [41]:
# Now, we mask 20% of the customer/product interactions.
product_train, product_test, product_customers_altered = make_train(purchases_sparse, pct_test = 0.2)

### Implementing ALS for Implicit Feedback based collaborative filtering

In [42]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of confidence with shape m x n, where m is the number of customers and n is the number of products.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various confidence.
    
    iterations - The number of times to alternate between both customer feature vector and product feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the customer/product feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for customers and products. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_customer = conf.shape[0]
    num_product = conf.shape[1] # Get the size of our original confidence matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_customer, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_product, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_customer)
    Y_eye = sparse.eye(num_product)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_customer):
            conf_samp = conf[u,:].toarray() # Grab customer row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_product):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

In [43]:
#*check* this takes a long time
#*check still need to run with 15 iterations
customer_vecs, product_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1,
                                            rank_size = 20)

### We can investigate confidence for a particular customer by taking the dot product between the customer and product vectors.
### Let’s look at our first customer.

In [44]:
customer_vecs[0,:].dot(product_vecs).toarray()[0,:5]

array([5.24299219e-02, 9.85570586e-05, 1.25361284e-02, 3.89815739e-02,
       7.83943702e-03])

-This is a sample of the first five items out of the 158 in our stock. The first user in our matrix has the second item with the greatest recommendation out of the first five items. However, notice we only did one iteration because our algorithm was so slow! You should iterate at least ten times according to the authors so that U and V converge.
This is done using matrix factorization

## Making ALS faster

In [45]:
# As our algorithm is very parallel, we can use ALS for Python utilizing Cython and parallelizing the code among threads.
import implicit

In [46]:

alpha = 15
conf_data = (product_train*alpha).astype('double')
customer_vecs, product_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


  0%|          | 0/50 [00:00<?, ?it/s]

## Evaluation of Recommendation Engine

- Remember that our training set had 20% of the purchases masked? This will allow us to evaluate the performance of our recommender system. Essentially, we need to see if the order of recommendations given for each user matches the items they ended up purchasing. A commonly used metric for this kind of problem is the area under the Receiver Operating Characteristic (or ROC) curve. A greater area under the curve means we are recommending items that end up being purchased near the top of the list of recommended items.

In [85]:
from sklearn import metrics
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   

- Now, utilize this helper function inside of a second function that will calculate the AUC for each user in our training set that had at least one item masked. It should also calculate AUC for the most popular items for our users to compare.

In [48]:
def calc_mean_auc(training_set, altered_customers, predictions, test_set):
    '''
    This function will calculate the mean AUC by customer for any customer that had their customer-product matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    customer/product interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted confidence for each customer/product pair as output from the implicit MF.
    These should be stored in a list, with customer vectors as product zero and product vectors as product one. 
    
    altered_customers - The indices of the customers where at least one customer/product pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on customer-product interactions
    there were originally zero to test ranking ability in addition to the most popular products as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each customer that had an product removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_products = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of product iteractions to find most popular
    product_vecs = predictions[1]
    for customer in altered_customers: # Iterate through each customer that had an product altered
        training_row = training_set[customer,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our customer/product vectors
        customer_vec = predictions[0][customer,:]
        pred = customer_vec.dot(product_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the products that were originally zero
        # Select all confidence from the MF prediction for this customer that originally had no iteraction
        actual = test_set[customer,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_products[zero_inds] # Get the product popularity for our chosen products
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given customer and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End customers iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [49]:
calc_mean_auc(product_train, product_customers_altered, 
              [sparse.csr_matrix(customer_vecs), sparse.csr_matrix(product_vecs.T)], product_test)
# AUC for our recommender system

(0.773, 0.836)

- First measure is the "collaborative filtering" method, where recommendation are based on similar products purchased by other customers
- Popularity measure is recommending the most popular products instead of "similar" products bought by other customers
- *check* in both cases these metrics are pretty good, reading AUC as a confidence level suggests the recommendations are in general what the customer is buying most of the time, but the fact the recommending most popular products beats
- similar product recommendations suggests collaborative filtering in this case doesnt have any business value 

In [50]:
customers_arr = np.array(customers) # Array of customer IDs from the confidence matrix
products_arr = np.array(products) # Array of product IDs from the confidence matrix

In [51]:
#some checks as we havent looked for a while
#some high level stats
customers_arr.shape

(18484,)

In [52]:
products_arr.shape

(158,)

In [53]:
def get_products_purchased(customer_id, mf_train, customers_list, products_list, product_lookup):
    '''
    This just tells us which products have been already purchased by a specific customer in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial confidence training set used (without weights applied)
    
    customers_list - The array of customers used in the confidence matrix
    
    products_list - The array of products used in the confidence matrix
    
    product_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of product IDs and product descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased products
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased products
    return product_lookup.loc[product_lookup.ProductID.isin(prod_codes)]

In [54]:
customers_arr[:5]

array([11000, 11001, 11002, 11003, 11004], dtype=int64)

In [55]:
# Our first customer in the list has an id of 11000. Let's test the function.
get_products_purchased(11000, product_train, customers_arr, products_arr, product_lookup)

,ProductID,ProductName
5432,353,"Mountain-200 Silver, 38"
5442,214,"Sport-100 Helmet, Red"
5457,541,Touring Tire
5458,530,Touring Tire Tube
5459,573,"Touring-1000 Blue, 46"
5476,485,Fender Set - Mountain


- We can see that the customer purchased 6 items from the store. What items does the recommender system say this customer should purchase? We need to create another function that does this. Let’s also import the MinMaxScaler from scikit-learn to help with this.

In [56]:
from sklearn.preprocessing import MinMaxScaler
def rec_products(customer_id, mf_train, customer_vecs, product_vecs, customer_list, product_list, product_lookup, num_products = 10):
    '''
    This function will return the top recommended products to our customers 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    customer_vecs - the customer vectors from your fitted matrix factorization
    
    product_vecs - the product vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your confidence matrix 
                    (in order of matrix)
    
    product_list - an array of the products that make up the columns of your confidence matrix
                    (in order of matrix)
    
    product_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_products - The number of products you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the customer/product vectors for products never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the confidence from the training set confidence matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that products not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = customer_vecs[cust_ind,:].dot(product_vecs.T) # Get dot product of customer vector and all product vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # products already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_products] # Sort the indices of the products into order 
    # of best recommendations
    rec_list = [] # start empty list to store products
    for index in product_idx:
        code = product_list[index]
        rec_list.append([code, product_lookup.ProductName.loc[product_lookup.ProductID == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [product[0] for product in rec_list]
    descriptions = [product[1] for product in rec_list]
    final_frame = pd.DataFrame({'ProductID': codes, 'ProductName': descriptions}) # Create a dataframe 
    return final_frame[['ProductID', 'ProductName']] # Switch order of columns around


- Essentially, this will retrieve the N highest ranking dot products between our user and item vectors for a particular user. Items already purchased are not recommended to the user. For now, let’s use a default of 3 items and see what the recommender system decides to pick for our customer.

In [58]:
# Let's test the function
DisplayRecommendation = rec_products(11000, product_train, customer_vecs, product_vecs, customers_arr, products_arr, product_lookup,
                       num_products = 5)

print("The top two recommended products are: \n",   
     DisplayRecommendation.iloc[0,1],
     "and \n",
     DisplayRecommendation.iloc[1,1])

DisplayRecommendation['ProductID']=DisplayRecommendation['ProductID'].astype(int) #switch ProductID to integer to join tables
DisplayRecommendation=DisplayRecommendation.merge(allproducts[['ProductID','EnglishProductSubcategoryName']],how="left",on='ProductID')

display(DisplayRecommendation)


The top two recommended products are: 
 Long-Sleeve Logo Jersey, S and 
 Mountain-200 Black, 38


,ProductID,ProductName,EnglishProductSubcategoryName
0,228,"Long-Sleeve Logo Jersey, S",Jerseys
1,358,"Mountain-200 Black, 38",Mountain Bikes
2,362,"Mountain-200 Black, 46",Mountain Bikes
3,357,"Mountain-200 Silver, 46",Mountain Bikes
4,361,"Mountain-200 Black, 42",Mountain Bikes


## Recommendation Graphic User Interface

In [61]:
from IPython.display import display_html
def Recommendation():
    Customer_ID=int(input("Customer ID:"))
    df1=(get_products_purchased(Customer_ID, product_train, customers_arr, products_arr, product_lookup))
    df2=(rec_products(Customer_ID, product_train, customer_vecs, product_vecs, customers_arr, products_arr, product_lookup,
                       num_products = 5))
    
    #add product subcategory to recommendation table
    df2['ProductID']=df2['ProductID'].astype(int) #switch ProductID to integer to join tables
    df2=df2.merge(allproducts[['ProductID','EnglishProductSubcategoryName']],how="left",on='ProductID')

    
    df1_styler = df1.style.\
                set_table_attributes("style='display:inline'").\
                set_caption('This customer has purchased:')
    df2_styler = df2.style.\
                    set_table_attributes("style='display:inline'").\
                    set_caption('This customer should like:')
    display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)
    
    print("So the top two recommended products are: \n",   
     df2.iloc[0,1],
     "and \n",
     df2.iloc[1,1])

In [62]:
Recommendation()

Customer ID:11000


This customer has purchased: 
 
 
   
 ProductID 
 ProductName 
 
 
 
 
 5432 
 353 
 Mountain-200 Silver, 38 
 
 
 5442 
 214 
 Sport-100 Helmet, Red 
 
 
 5457 
 541 
 Touring Tire 
 
 
 5458 
 530 
 Touring Tire Tube 
 
 
 5459 
 573 
 Touring-1000 Blue, 46 
 
 
 5476 
 485 
 Fender Set - Mountain 
 
 
 

 
 This customer should like: 
 
 
   
 ProductID 
 ProductName 
 EnglishProductSubcategoryName 
 
 
 
 
 0 
 228 
 Long-Sleeve Logo Jersey, S 
 Jerseys 
 
 
 1 
 358 
 Mountain-200 Black, 38 
 Mountain Bikes 
 
 
 2 
 362 
 Mountain-200 Black, 46 
 Mountain Bikes 
 
 
 3 
 357 
 Mountain-200 Silver, 46 
 Mountain Bikes 
 
 
 4 
 361 
 Mountain-200 Black, 42 
 Mountain Bikes

So the top two recommended products are: 
 Long-Sleeve Logo Jersey, S and 
 Mountain-200 Black, 38


### Commercial Value

- Lets assume from the top 10 recommendations presented to each customer, 1 item is bought each year and costs the average 
- price of the top 10 recommendations

- We need to find the cost of the top 10 recommendations first

In [64]:
CustomerRecommendation = rec_products(11000, product_train, customer_vecs, product_vecs, customers_arr, products_arr, product_lookup,
                       num_products = 10)

In [65]:
CustomerRecommendation

,ProductID,ProductName
0,228,"Long-Sleeve Logo Jersey, S"
1,358,"Mountain-200 Black, 38"
2,362,"Mountain-200 Black, 46"
3,357,"Mountain-200 Silver, 46"
4,361,"Mountain-200 Black, 42"
5,352,"Mountain-200 Silver, 38"
6,478,Mountain Bottle Cage
7,487,Hydration Pack - 70 oz.
8,356,"Mountain-200 Silver, 46"
9,576,"Touring-1000 Blue, 60"


In [66]:
allproducts.head()

,ProductID,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,...,JapaneseDescription,TurkishDescription,StartDate,EndDate,Status,ProductSubcategoryAlternateKey,EnglishProductSubcategoryName,SpanishProductSubcategoryName,FrenchProductSubcategoryName,ProductCategoryKey
0,1,AR-5381,NaN,None,None,Adjustable Race,,,NaN,False,...,None,None,2003-07-01,NaT,Current,NaN,NaN,NaN,NaN,NaN
1,2,BA-8327,NaN,None,None,Bearing Ball,,,NaN,False,...,None,None,2003-07-01,NaT,Current,NaN,NaN,NaN,NaN,NaN
2,3,BE-2349,NaN,None,None,BB Ball Bearing,,,NaN,False,...,None,None,2003-07-01,NaT,Current,NaN,NaN,NaN,NaN,NaN
3,4,BE-2908,NaN,None,None,Headset Ball Bearings,,,NaN,False,...,None,None,2003-07-01,NaT,Current,NaN,NaN,NaN,NaN,NaN
4,5,BL-2036,NaN,None,None,Blade,,,NaN,False,...,None,None,2003-07-01,NaT,Current,NaN,NaN,NaN,NaN,NaN


In [67]:
#Import Factinternetsales
import pandas as pd
fctintsales = pd.read_sql_query("""
SELECT
[ProductKey]
,avg([SalesAmount]) AS Avg_Sales
,max([SalesAmount]) AS Max_Sales
FROM [dbo].[FactInternetSales] group by [ProductKey] """, conn)

In [68]:
#just get back product cost and list price
products_short=fctintsales

In [69]:
#rename ProductKey as ProductID as need to join tables
products_short=products_short.rename(columns={"ProductKey": "ProductID"})

In [70]:
#take a look
products_short.head()

,ProductID,Avg_Sales,Max_Sales
0,593,564.99,564.99
1,355,2319.99,2319.99
2,570,742.35,742.35
3,378,2443.35,2443.35
4,384,1120.49,1120.49


In [71]:
products_short.dtypes

ProductID      int64
Avg_Sales    float64
Max_Sales    float64
dtype: object

In [72]:
CustomerRecommendation.dtypes

ProductID      object
ProductName    object
dtype: object

In [73]:
#need to make productID same type in both tables
CustomerRecommendation['ProductID'] = CustomerRecommendation.ProductID.astype(int)

In [74]:
CustomerRecommendation=CustomerRecommendation.merge(products_short,how="left",on='ProductID')

In [75]:
CustomerRecommendation

,ProductID,ProductName,Avg_Sales,Max_Sales
0,228,"Long-Sleeve Logo Jersey, S",49.9900,49.9900
1,358,"Mountain-200 Black, 38",2049.0982,2049.0982
2,362,"Mountain-200 Black, 46",2049.0982,2049.0982
3,357,"Mountain-200 Silver, 46",2319.9900,2319.9900
4,361,"Mountain-200 Black, 42",2294.9900,2294.9900
5,352,"Mountain-200 Silver, 38",2071.4196,2071.4196
6,478,Mountain Bottle Cage,9.9900,9.9900
7,487,Hydration Pack - 70 oz.,54.9900,54.9900
8,356,"Mountain-200 Silver, 46",2071.4196,2071.4196
9,576,"Touring-1000 Blue, 60",2384.0700,2384.0700


In [76]:
#assumed additional income from customer as result of recommendation 
#= average cost and list price for the customer's top 10 recommendations
#CustomerRecommendation.StandardCost.mean()
print("based on product Standard cost ", CustomerRecommendation.Avg_Sales.mean())
print("based on product List price", CustomerRecommendation.Max_Sales.mean())

based on product Standard cost  1535.50556
based on product List price 1535.50556


In [77]:
internet_sales.CustomerID.nunique()

18484

In [78]:
#too many customers to loop thru, so lets look at top 500 customers by order volume only

#first group and sort the customers
rankedcust=internet_sales.groupby('CustomerID').agg({'OrderVol':'sum'}).sort_values(by='OrderVol', ascending=False).reset_index()

In [79]:
rankedcust.head()

,CustomerID,OrderVol
0,11185,68
1,11300,67
2,11277,65
3,11262,63
4,11287,62


In [80]:
#check
rankedcust.shape

(18484, 2)

In [81]:
Benefit_SC=0
Benefit_LP=0

for i in range (1,500):
    #top 10 recommendations for customer
    CustomerRecommendation = rec_products(rankedcust.CustomerID[i-1], product_train, customer_vecs, product_vecs, customers_arr, products_arr, product_lookup,
                           num_products = 10)

    #need to make productID same type in both tables
    CustomerRecommendation['ProductID'] = CustomerRecommendation.ProductID.astype(int)
    CustomerRecommendation=CustomerRecommendation.merge(products_short,how="left",on='ProductID')

    Benefit_SC=Benefit_SC+CustomerRecommendation.Avg_Sales.mean()
    Benefit_LP=Benefit_LP+CustomerRecommendation.Max_Sales.mean()

print("Commercial Benefit (based on Standard Cost) if all customers buy one of their top 10 recommended purchases", Benefit_SC)
print("Commercial Benefit (based on List Price) if all customers buy one of their top 10 recommended purchases", Benefit_LP)  

Commercial Benefit (based on Standard Cost) if all customers buy one of their top 10 recommended purchases 388050.4043499999
Commercial Benefit (based on List Price) if all customers buy one of their top 10 recommended purchases 388050.4043499999
